In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
from ebooklib import epub

def next_page(next_text, soup):
    next_link = soup.find("a", string=next_text)
    next_link = base_url + next_link["href"]
    return next_link

def extract_book_title(soup):
    title = soup.find(book_title_tag)
    return title.text
    
def extract_chapter_title(soup):
    title = soup.find(chapter_title_tag)
    return title.text

def extract_chapter(soup):
    if content_tag[0] == "id":
        content = soup.find(id=content_tag[1])
    else:
        content = soup.find(content_tag[0], class_=content_tag[1])
        
    content = content.get_text(separator="\n\n")
    for string in to_be_replaced:
        content = content.replace(string, "")
    return content

def run_test():
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    next_url = next_page(next_text, soup)
    print(next_url)
    
    book_title = extract_book_title(soup)
    print(book_title)
    
    chapter_title = extract_chapter_title(soup)
    print(chapter_title)
    
    content = extract_chapter(soup)
    print(content[:100])

# Config

In [ ]:
# url
url = "https://sj.uukanshu.com/read.aspx?tid=131112&sid=86521"
base_url = "https://sj.uukanshu.com/"
next_text = "[下一章]"

# css
book_title_tag = "h1"
chapter_title_tag = "h3"
content_tag = ["id", "bookContent"]

# optional
language = 'zh'
file_format = "epub"
count = 0
to_be_replaced = ["UU看書www.uukａnｓhu.ｃｏｍ"]

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [ ]:
run_test()

# Save as epub

In [ ]:
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
continue_extract = True
counter = 1
book_title = extract_book_title(soup)

if file_format == 'epub':
    toc = []
    spine = []
    book = epub.EpubBook()
    book.set_title(book_title)
    book.set_language(language)
else:
    filename = book_title + ".txt"
    f = open(filename, "a+")

while url is not None and continue_extract:
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    time.sleep(random.randint(3, 6))

    chapter_title = extract_chapter_title(soup)
    content = extract_chapter(soup)
    print(chapter_title)
    
    # save as epub or txt file
    if file_format == 'epub':
        # create chapter
        chap = epub.EpubHtml(title=chapter_title, file_name=chapter_title + '.xhtml')
        chap.content = content.replace('\n', '<br>')

        # add chapter
        book.add_item(chap)
        spine.append(chap)

        # create toc
        link = epub.Link(chap.file_name, chap.title, chap.id)
        toc.append(link)
    else:
        chapter = ""
        chapter = chapter_title + "\n\n\n\n"
        chapter = chapter + content
        f.write(chapter)
    
    # break if more than count
    if count == 0:
        pass
    elif counter < count:
        counter += 1
        pass
    else:
        break

    # get next url
    try:
        next_url = next_page(next_text, soup)
        url = next_url
    except:
        print("End of url")
        break
        
if file_format == 'epub':
    # toc
    toc = tuple(toc)
    book.toc = toc

    # add default NCX and Nav file
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # spine = tuple(spine)
    book.spine = ['nav', *spine]

    # write to the file
    epub.write_epub(book_title + '.epub', book, {})
else:
    f.close()
    
print("\n=====\nDONE")